# Naive Bayes Model

In this notebook, a Naive Bayes model is run on a iid sampled data set of approximately 670K rows of data.  This notebook was run on an AWS SageMaker ml.c5.4xlarge instance.

#### Import modules

In [38]:
import pandas as pd
import string
import re
import string
import numpy as np
import datetime

import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer 

import warnings
warnings.filterwarnings('ignore')

import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.naive_bayes import MultinomialNB
from sklearn import svm
from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

In [39]:
import feature_generation_functions as fgf

In [40]:
import model_functions as mf

In [41]:
import pickle_functions as pf

### Load and shuffle data

In [5]:
train = pd.read_csv("s3://advancedml-koch-mathur-hinkson/train.csv")

In [6]:
train.shape

(1804874, 45)

Label comments as toxic ("1") or nontoxic ("0") using 0.5 threshold

In [7]:
train['toxicity_category'] = train.target.apply(lambda x: 1 if x > 0.5 else 0)

In [8]:
train.shape

(1804874, 46)

Split into train_set and validation_set

In [9]:
#Citation: https://stackoverflow.com/questions/24147278/how-do-i-create-test-and-train-samples-from-one-dataframe-with-pandas
msk = np.random.rand(len(train)) < 0.8
train_set = train[msk]
validation_set = train[~msk]

In [14]:
print(train_set.toxicity_category.value_counts())

0    1358850
1      85018
Name: toxicity_category, dtype: int64


In [15]:
print(validation_set.toxicity_category.value_counts())

0    339586
1     21420
Name: toxicity_category, dtype: int64


Randomly sample train_set to create a smaller data frame (train_sample) to run SVM on

In [16]:
train_sample = train_set.sample(frac=0.5, replace=True)

In [17]:
print(train_sample.toxicity_category.value_counts())

0    679613
1     42321
Name: toxicity_category, dtype: int64


### Generate features

In [18]:
train_df = fgf.generate_NB_SVM_features(train_sample)

Cleaned with stopwords...Elapsed Time:  0.177 minutes
Cleaned without stopwords...Elapsed Time:  0.245 minutes
Stemmed (Porter)...Elapsed Time:  5.661 minutes
Stemmed (Lancaster)...Elapsed Time:  4.641 minutes

DONE GENERATING FEATURES


In [20]:
pf.write_pickle_to_s3bucket(filename='NB_final_720K', 
                            bucket_name='advancedml-koch-mathur-hinkson', 
                            df=train_df)

Pickled and sent to bucket!


In [35]:
validation_df = fgf.generate_NB_SVM_features(validation_set)

Cleaned with stopwords...Elapsed Time:  0.088 minutes
Cleaned without stopwords...Elapsed Time:  0.122 minutes
Stemmed (Porter)...Elapsed Time:  2.797 minutes
Stemmed (Lancaster)...Elapsed Time:  2.332 minutes

DONE GENERATING FEATURES


In [ ]:
pf.write_pickle_to_s3bucket(filename='NB_final_720K', 
                            bucket_name='advancedml-koch-mathur-hinkson', 
                            df=validation_df)

### Reshaping

In [ ]:
# train_df = pf.read_pickle(filename='NB_final_720K', bucket_name='advancedml-koch-mathur-hinkson')

In [30]:
toxic = train_df[train_df.toxicity_category == 1]
nontoxic = train_df[train_df.toxicity_category == 0]

In [31]:
train_df.shape, toxic.shape, nontoxic.shape

((721934, 50), (42321, 50), (679613, 50))

Reshape the dataset to include an equal number of toxic and nontoxic samples

In [32]:
quarter = len(toxic)

In [33]:
random_df = train_df.sample(quarter*4)

In [34]:
prepared_25 = toxic.append(nontoxic.sample(len(toxic)*3))
prepared_25 = prepared_25.sample(frac=1).reset_index(drop=True)
print(prepared_25.toxicity_category.value_counts())

prepared_50 = toxic.append(toxic).append(nontoxic.sample(len(toxic)*2))
prepared_50 = prepared_50.sample(frac=1).reset_index(drop=True)
print(prepared_50.toxicity_category.value_counts())

prepared_75 = toxic.append(toxic).append(toxic).append(nontoxic.sample(len(toxic)))
prepared_75 = prepared_75.sample(frac=1).reset_index(drop=True)
print(prepared_75.toxicity_category.value_counts())

0    126963
1     42321
Name: toxicity_category, dtype: int64
1    84642
0    84642
Name: toxicity_category, dtype: int64
1    126963
0     42321
Name: toxicity_category, dtype: int64


### Naive Bayes

In [42]:
best_metric = 0
metric_dict = ''
model_factors = []

SUBSET_OF_INTEREST = "Target"
METRIC_OF_INTEREST = "F1"

dfs = [random_df, prepared_50]
label = ["random_df", "prepared_50"]

mn = 0

for i in range(len(dfs)):
    for text in ['cleaned_w_stopwords_str', 'cleaned_no_stem_str', 'cleaned_porter_str',
       'cleaned_lancaster_str']:

        factors = [label[i], text]
        mn += 1
        print("{}. {}".format(mn, datetime.datetime.now()))
        print(factors)

        clf, output = mf.run_model(dfs[i], comments = text, model_type = "MultiNB")
        metrics = mf.get_metrics(output, should_print=False)
        metric_of_interest = metrics[SUBSET_OF_INTEREST][METRIC_OF_INTEREST]
        
        print("Overall Accuracy: {}, Target Accuracy: {}, Non-Target Accuracy: {}".format(metrics["Overall"]["Accuracy"], metrics["Target"]["Accuracy"], metrics["Non-Target"]["Accuracy"]))
        print() 
        
        if (metric_of_interest > best_metric) and metric_of_interest < 0.95:
            best_metric = metric_of_interest
            
            model_factors = factors
            metric_dict = metrics

1. 2019-05-30 19:06:00.179089
['random_df', 'cleaned_w_stopwords_str']


NameError: name 'TfidfVectorizer' is not defined